In [1]:
import Data.List
import Data.Traversable
import Database.HDBC
import Database.HDBC.Sqlite3

In [2]:
db <- connectSqlite3 "../data/enron.sqlite3"

In [9]:
quickQuery db "SELECT count(*) FROM document" []

[[SqlInt64 5172]]

In [10]:
quickQuery db "SELECT total(frequency) FROM word" []

[[SqlDouble 850027.0]]

In [11]:
quickQuery db "SELECT total(frequency) FROM word WHERE word='website'" []

[[SqlDouble 139.0]]

In [12]:
pWebsite = 139 / 850027

In [13]:
pWebsite

1.6352421746603342e-4

In [14]:
quickQuery db "SELECT total(frequency) FROM word,document WHERE word='website' AND word.document_id=document.document_id AND label='ham'" []

[[SqlDouble 74.0]]

In [15]:
quickQuery db "SELECT total(frequency) FROM word,document WHERE word.document_id=document.document_id AND label='ham'" []

[[SqlDouble 582265.0]]

In [16]:
pWebsiteGivenHam = 74/582265

In [17]:
pWebsiteGivenHam

1.2708989892918173e-4

In [18]:
quickQuery db "SELECT count(*) FROM document WHERE label='ham'" []

[[SqlInt64 3672]]

In [19]:
pHam = 3672 / 5172

In [20]:
pHam

0.7099767981438515

In [21]:
pHamGivenWebsite = (pHam * pWebsiteGivenHam) / pWebsite

In [22]:
pHamGivenWebsite

0.5517890922603469

In [23]:
quickQuery db "SELECT label FROM document WHERE document_id=8" []

[[SqlByteString "spam"]]

In [24]:
quickQuery db "SELECT word FROM word WHERE document_id=8" []

[[SqlByteString "and"],[SqlByteString "via"],[SqlByteString "secure"],[SqlByteString "ogrg"],[SqlByteString "soma"],[SqlByteString "is"],[SqlByteString "direct"],[SqlByteString "one"],[SqlByteString "ultram"],[SqlByteString "cost"],[SqlByteString "thanks"],[SqlByteString "ready"],[SqlByteString "our"],[SqlByteString "ship"],[SqlByteString "your"],[SqlByteString "doorstep"],[SqlByteString "click"],[SqlByteString "take"],[SqlByteString "medications"],[SqlByteString "for"],[SqlByteString "no"],[SqlByteString "please"],[SqlByteString "shipped"],[SqlByteString "fast"],[SqlByteString "pharmacy"],[SqlByteString "write"],[SqlByteString "to"],[SqlByteString "low"],[SqlByteString "online"],[SqlByteString "you"],[SqlByteString "more"],[SqlByteString "physicians"],[SqlByteString "door"],[SqlByteString "adipex"],[SqlByteString "oxwq"],[SqlByteString "here"],[SqlByteString "me"],[SqlByteString "a"],[SqlByteString "e"],[SqlByteString "lnu"],[SqlByteString "licensed"],[SqlByteString "approved"],[SqlBy

In [25]:

bayesWordLabel :: Connection -> String -> String -> IO Double
bayesWordLabel db word label = do
    fLabel <- sumFrequencyLabel label
    fWordGivenLabel <- sumFrequencyWordLabel word label
    let pWordGivenLabel = (decipher fWordGivenLabel) / (decipher fLabel)
    if pWordGivenLabel == 0 then
        return 0.00001
      else
        return pWordGivenLabel
  where
      sumFrequencyWordLabel word label = query ("SELECT total(frequency) FROM word, document WHERE word.document_id=document.document_id AND word.word='"++word++"' AND document.label='"++label++"'")
      sumFrequencyLabel label = query ("SELECT total(frequency) FROM word, document WHERE word.document_id=document.document_id AND document.label='"++label++"'")
      decipher x = fromIntegral (fromSql (head (head x)) :: Integer)
      query q = quickQuery db q []

In [26]:
bayesWordLabel db "website" "ham"

1.2708989892918173e-4

In [27]:
bayesWordLabel db "website" "spam"

2.4275289249408056e-4

In [28]:
0.29 * 2.4275289249408056e-4

7.039833882328335e-5

In [29]:
0.71 * 1.2708989892918173e-4

9.023382823971902e-5

In [30]:

bayesDocumentLabel :: Connection -> String -> String -> IO Double
bayesDocumentLabel db docid label = do
    cAll <- countAll
    cLabel <- countLabel label
    let pLabel = (decipher cLabel) / (decipher cAll)
    docWords <- wordsIO
    probs <- traverse (\word -> bayesWordLabel db word label) (pullWords docWords)
    return $ pLabel * foldr (*) 1.0 probs
  where
      countAll = query "SELECT count(*) FROM document"
      countLabel label = query ("SELECT count(*) FROM document WHERE label='"++label++"'")
      sumFrequencyWord word = query ("SELECT total(frequency) FROM word WHERE word.word='"++word++"'")
      wordsIO = query ("SELECT word FROM word WHERE document_id="++docid)
      pullWords dw = map fromSql (head (transpose dw))
      decipher x = fromIntegral (fromSql (head (head x)) :: Integer)
      query q = quickQuery db q []

In [31]:
bayesDocumentLabel db "8" "ham"

4.098748931287863e-223

In [32]:
bayesDocumentLabel db "8" "spam"

5.2669163791267e-198